In [2]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load Point Cloud


In [11]:
# (a) load point cloud
terrain_pc = o3d.io.read_point_cloud("./terrain.ply")
terrain_pc_noisy = o3d.io.read_point_cloud("./terrain_noisy.ply")
terrain_pc_partial = o3d.io.read_point_cloud("./terrain_partial.ply")

# (b) uniformly downsample the point cloud
terrain_pc_down = terrain_pc.uniform_down_sample(10)
terrain_pc_noisy_down = terrain_pc_noisy.uniform_down_sample(10)
terrain_pc_partial_down = terrain_pc_partial.uniform_down_sample(10)

print(terrain_pc_down)
print(terrain_pc_noisy_down)
print(terrain_pc_partial_down)


PointCloud with 143070 points.
PointCloud with 140117 points.
PointCloud with 140117 points.


In [12]:
# (c) visualize the downsampled point cloud
o3d.visualization.draw_geometries([terrain_pc_down])
o3d.visualization.draw_geometries([terrain_pc_noisy_down])
o3d.visualization.draw_geometries([terrain_pc_partial_down])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Denoise the point cloud

In [13]:
# (d) denoise the point cloud 

# print("terrain_pc_noisy:", terrain_pc_noisy, "terrain_pc_partial:",terrain_pc_partial)

# Apply statistical outlier removal
terrain_pc_denoised_down, _ = terrain_pc_noisy_down.remove_statistical_outlier(nb_neighbors=10, std_ratio=1.0)
# print("terrain_denoised:", terrain_pc_denoised_down)

# calculate the chamfer distance (quantify how good the denoised pc)


### Calculate Chamfer Distance Between Two Point Clouds


In [14]:

from scipy.spatial import KDTree

def chamfer_distance(pc1, pc2):
    """
    Compute the Chamfer distance between two point clouds using KD-tree for efficient nearest neighbor search.
    
    Parameters:
    - pc1, pc2: open3d.geometry.PointCloud objects
    
    Returns:
    - float, the Chamfer distance between pc1 and pc2
    """
    # Convert Open3D PointCloud to numpy arrays
    pc1_points = np.asarray(pc1.points)
    pc2_points = np.asarray(pc2.points)

    # Create a KD-Tree for the second point cloud
    tree2 = KDTree(pc2_points)

    # Find the nearest neighbor in pc2 for each point in pc1
    # min x-y
    distances, _ = tree2.query(pc1_points) # Query the kd-tree for nearest neighbors.
    # print(distances)
    # Compute the one-sided Chamfer distance
    chamfer_dist = np.mean(distances ** 2)
    
    return chamfer_dist


In [17]:

# Compute the Chamfer distance
distance1 = chamfer_distance(terrain_pc_denoised_down, terrain_pc_down)
print(f"The Chamfer distance between the denoised pc and ground-truth is: {distance1}")

distance2 = chamfer_distance(terrain_pc_noisy_down, terrain_pc_down)
print(f"The Chamfer distance between the noisy pc and ground-truth is: {distance2}")

distance2 = chamfer_distance(terrain_pc_partial_down, terrain_pc_down)
print(f"The Chamfer distance between the partial pc and ground-truth is: {distance2}")


The Chamfer distance between the denoised pc and ground-truth is: 0.00040969444983181166
The Chamfer distance between the noisy pc and ground-truth is: 0.0006414638119259131
The Chamfer distance between the partial pc and ground-truth is: 0.00020744294757761195
